In [16]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler
import joblib

In [17]:
import numpy as np
import pandas as pd

preprocessed_files = ["../Preprocesadas/Preprocessed_Data_Chunk_10_seg_17-04-2024.csv","../Preprocesadas/Preprocessed_Data_Chunk_20_seg_17-04-2024.csv","../Preprocesadas/Preprocessed_Data_Chunk_30_seg_17-04-2024.csv"]

emotions = ["Focus", "Amusement", "Surprise", "Fright", "Inmersion"]
models = ["MLP", "KNN", "DecisionTree", "MLP", "SVM", "RandomForest"]
hardwares = ["EEG_Emotibit","EEG", "Emotibit"]
chunks = ["chunk_10_seg", "chunk_20_seg", "chunk_30_seg"]

In [18]:
#El modelo no converge ni aun con 100000 iteraciones,
#pero el valor no cambia mucho.
#Aumentar max_iter cuando sea una prueba importante.
def SVM(X_train, y_train):
    lsvr_classifier = LinearSVR(dual=True, max_iter=10000)
    lsvr_classifier.fit(X_train, y_train)
    return lsvr_classifier

In [19]:
#Grid_search 
#stocastic random search

def MLP(X_train, y_train):
    mlp = MLPRegressor(hidden_layer_sizes=(1024, 512, 256, 128, 64), activation='relu', max_iter=7000, alpha=0.01, learning_rate_init=0.07, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp

In [20]:
def RandomForest(X_train, y_train):
   random_forest = RandomForestRegressor(random_state=42)
   random_forest.fit(X_train, y_train)
   return random_forest

In [21]:
def DecisionTree(X_train, y_train):
    knn = KNeighborsRegressor(n_neighbors=5)
    knn.fit(X_train, y_train)
    return knn

In [22]:
def KNN(X_train, y_train):
    decision_tree = DecisionTreeRegressor(random_state=42)
    decision_tree.fit(X_train, y_train)
    return decision_tree

## SOLO SPLIT

In [23]:

pd.set_option("display.max_columns", None)
pd.set_option("display.max_columns", 20)

i = 0

for preprocessed_file in preprocessed_files:

    Raw_Preprocesed = pd.read_csv(preprocessed_file)

    for model in models:

        for hardware in hardwares:

            if hardware == "EEG_Emotibit":
                # X con EEG y Emotibit (todo menos las emociones)
                X = Raw_Preprocesed.drop(emotions, axis=1)

            if hardware == "EEG":
                # X con solo EEG
                X = Raw_Preprocesed.loc[:, :'AX_t0.000'].iloc[:, :-1]

            if hardware == "Emotibit":
                # X con solo Emotibit
                X = Raw_Preprocesed.loc[:, 'AX_t0.000':'Focus'].iloc[:, :-1]
            
            y = Raw_Preprocesed[emotions]

            scaler = StandardScaler()
            X = scaler.fit_transform(X)

            #Entrenamos un modelo distinto por emoción.
            for emotion in emotions:
                print(f"--- Modelo {model} para {emotion} con hardware {hardware} y {chunks[i]} ---")

                X_train, X_test, y_train, y_test = train_test_split(X, y[emotion], test_size=0.25, random_state=8)

                classifier = any

                if model == "SVM":
                    classifier = SVM(X_train, y_train)

                elif model == "MLP":
                    classifier = MLP(X_train, y_train)

                elif model == "RandomForest":
                    classifier = RandomForest(X_train, y_train)

                elif model == "DecisionTree":
                    classifier = DecisionTree(X_train, y_train)

                elif model == "KNN":
                    classifier = KNN(X_train, y_train)

                # Guarda el modelo entrenado en un archivo
                model_filename = f"{model}_{emotion}_{hardware}_{chunks[i]}.joblib"
                joblib.dump(classifier, model_filename)
                print(f"Modelo para {emotion} guardado en {model_filename}")

                predictions = classifier.predict(X_test)

                mse = mean_squared_error(y_test, predictions)
                print(f"Mean squared error: {mse:.4f}")

                r2 = r2_score(y_test, predictions)
                print(f"R-squared: {r2:.4f}")

                print(f"Real {emotion}:")
                display(pd.DataFrame(y_test).T)

                print(f"{emotion} prediction:")
                display(pd.DataFrame(predictions).T)

                print('\n')
    
    i += 1

--- Modelo MLP para Focus con hardware EEG_Emotibit y chunk_10_seg ---
Modelo para Focus guardado en MLP_Focus_EEG_Emotibit_chunk_10_seg.joblib
Mean squared error: 0.7005
R-squared: -0.4445
Real Focus:


,189,25,202,22,100,93,273,277,164,34,...,248,238,80,220,175,23,178,111,84,42
Focus,-1.0,1.0,0.5,0.5,0.5,0.5,1.0,0.5,0.0,1.0,...,1.0,1.0,0.5,0.5,-1.0,0.5,-1.0,0.5,0.5,0.5


Focus prediction:


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,...,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554,-0.179554




--- Modelo MLP para Amusement con hardware EEG_Emotibit y chunk_10_seg ---
Modelo para Amusement guardado en MLP_Amusement_EEG_Emotibit_chunk_10_seg.joblib
Mean squared error: 173665363500.7394
R-squared: -527173909756.1851
Real Amusement:


...

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596,...,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596,0.365596




--- Modelo MLP para Inmersion con hardware EEG_Emotibit y chunk_10_seg ---
Modelo para Inmersion guardado en MLP_Inmersion_EEG_Emotibit_chunk_10_seg.joblib
Mean squared error: 13482827001.7673
R-squared: -37337059388.5095
Real Inmersion:


,189,25,202,22,100,93,273,277,164,34,...,248,238,80,220,175,23,178,111,84,42
Inmersion,-1.0,1.0,0.0,1.0,1.0,1.0,0.5,0.0,0.0,1.0,...,0.5,0.0,0.5,0.0,-1.0,1.0,-1.0,1.0,1.0,0.5


Inmersion prediction:


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,-46878.521564,-33534.037896,-42202.276993,-39809.794974,-213297.538925,-131413.087151,-46946.160907,-60644.829929,-62921.466197,-18896.784268,...,-105557.601092,-114451.849844,-101757.152667,-48565.327632,-175038.157465,-48799.367748,-183781.228072,-201022.782757,-182570.701855,-53012.733463
